In [ ]:
# Below we have sections of short driver scripts that run different parts of the heatmap code
# Not all functions in the 5 functiond definition .py's are called yet,
# but the ones that are called so far compile below without error

#%load_ext autoreload
#%autoreload 2

In [ ]:
import importlib
import setup_nx # your own module, setup.nx.py
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
import xlrd
from operator import add
importlib.reload(setup_nx)
from setup_nx import *
from graphviz import Source, render
import pprint

import datetime
import time

import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm


In [ ]:
# List functions in each helper .py
help(ff)
print("----------------------------------------------------------")
help(imp)
print("----------------------------------------------------------")
help(vis)
print("----------------------------------------------------------")
help(lzn)
print("----------------------------------------------------------")
help(hm)
print("----------------------------------------------------------")
help(ctrl)

In [ ]:
# [Essential] specify input feeder data

#-----------------------------------------------------------------
'specifying file paths'
# Enter the path/name of the model's excel file and import
    # All GridBright load files should be in the following folder
    #loadfolder = "/Users/jasperpakshong/Documents/Berkeley/ENERGISE/IEEE13/"
    #loadpath = loadfolder + "IEEE13testload_w_extreme_act.xlsx"

    # filepath = "IEEE13/"
    # modelpath = filepath + "001 phasor08_IEEE13_OPAL.xls"
    # loadfolder = "IEEE13/"
    # loadpath = loadfolder + "001_phasor08_IEEE13_norm03_HIL_7_1.xlsx"

    #filepath = "AL0001/"
    #modelpath = filepath + "AL0001_OPAL_working.xls"
    #loadfolder = "AL0001/"
    #loadpath = loadfolder + "AL0001_tvload_afternoon1h.csv"
    
    #filepath = "13NF_balanced/"
    #modelpath = filepath + "016 GB_IEEE13_balance_all_ver2.xls"
    #loadfolder = "13NF_balanced/"
    #loadpath = loadfolder + "016 GB_IEEE13_balance all ver2_time_sigBuilder_secondWise_norm03.csv"

filepath = "13NF_balanced/"
modelpath = filepath + "impedMod_IEEE13 unbal copy.xlsx" #"016 GB_IEEE13_balance_all_ver2.xls"
loadfolder = "13NF_balanced/"
loadfile = "016 GB_IEEE13_balance all ver2_time_sigBuilder_secondwise_norm03.csv"
loadpath = loadfolder + loadfile
headerpath = '13NF_balanced/001_phasor08_IEEE13_time_header.csv'
load_data = '13NF_balanced/001_phasor08_IEEE13_time_PVpen50_cloud_minutewise (1).csv'

#==========================================================================================================

'specifying file name'
#file_name = string specifying name of dot file created when make_graph() is called
file_name = '13NF_test.dot'

#==========================================================================================================

'list actuator locations and performance node'
# all_act_locs = list of node names as strings where actuators will be placed
all_act_locs = ['bus_651','bus_611','bus_633']
perf_nodes = ['bus_651','bus_611','bus_633']

#==========================================================================================================

'Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_650'
timesteps = 1

'DO NOT NEED TO EDIT THIS CELL BEFORE RUNNING'

ts = time.time()
print()
print(datetime.datetime.fromtimestamp(ts))

plot = 0 #turn plot on/off

depths = {}
leaves = []

In [ ]:
# --------- [ESSENTIAL] create feeder obj ----------
fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

print(depths) # should be populated

count = 0 # print list of buses in network
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
Source.from_file(file_name)
#^ need this to plot feeder

In [ ]:
# ----- Create Fig 1 ---------
# we have two options, both are not very interesting for 13NF

# Assess feas of placing 1 colocated at each node on feeder
# result: all locations are green
all_act_locs = []
perf_nodes = []
# empty act locs asks heatMapProcess to create heatmap for empty network, i.e. feas of placing 1 co-located actuator at each loc
fig1_feas_configs, heatMapNames=hm.find_good_colocated(fin_feeder, [], node_index_map,substation_name,depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
ff.clear_graph(fin_feeder)

#pprint.pprint(fig1_feas_configs)

In [ ]:
# Colormap showing good places to place colocated (find_good_coloc):
Source.from_file(heatMapNames[0]) # display graph in notebook

In [ ]:
print('----------- Feas configs are: ---------')

# Place maximum number of colocated act/perf nodes on network randomly. If infeas loc tested, randomly select another test node and continue function run
# result: can place colocated at all nodes
max_act_config = hm.place_max_coloc_acts(fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
# Actuator map showing the result of randomly placing coloc until cannot find feas loc (place_max_coloc_acts):
Source.from_file('actConfig_max_coloc')

In [ ]:
# ----- Create Figure 2 ---------
# phase coupling values = ratio of mutual impedance to self impedance for each path to substation
ratios = vis.phaseCouplingPerNode(fin_feeder, depths, file_name) # "a -->" b indicates interval (a,b)
vis.createColorMap(fin_feeder, ratios, 'fig2_13NF')
print('\n Phase Coupling Ratios')
print(ratios)
Source.from_file('colorMap_fig2_13NF') # display graph in notebook

In [ ]:
# ----- Create Figure 3 ---------
#  Color map overlayed onto feeder diagram that shows linearization error
# "feas_configs" should exist from call to "find_good_colocated" called for Fig 1
lzn_err_dic = {}
errType=0 # choose among 0,1,2,3 for [PV, QV, Pdel,Qdel] lzn errors
scale=100000 # so values nicely fit into bins
for dic in fig1_feas_configs:
    node = (dic['act'])[0]
    lzn_err = (dic['lznErr'])[0][errType] #extract lzn error scalar from fig1_feas_configs dict
    lzn_err_dic[node] = lzn_err*scale

pprint.pprint(lzn_err_dic)
vis.createColorMap(fin_feeder, lzn_err_dic, 'fig3_13NF') 
ff.clear_graph(fin_feeder)
Source.from_file('colorMap_fig3_13NF')  # display graph in notebook

In [ ]:
#pprint.pprint(fig1_feas_configs)
print(dic['lznErr'])
print((dic['lznErr'])[0][0])

In [ ]:
# ----- Create Table 1 ---------
# complex config 1
all_act_locs = ['bus_675','bus_611','bus_652','bus_645']
perf_nodes = ['bus_671','bus_684','bus_684','bus_632']

Zbase = (Vbase**2)/(Sbase*1000)
P_lb_results, P_ub_results, Q_lb_results, Q_ub_results = lzn.computePQsweep_timesteps(fin_feeder, load_data)
PQ_bounds, PQ_losses = lzn.computePQsweep_losses(fin_feeder, all_act_locs, Sbase, Zbase, P_lb_results, P_ub_results, Q_lb_results, Q_ub_results, headerpath, substation_name, modelpath, depths)
data = [['Timestep', 'Computed Line Losses', '', 'True Line Losses', ''],
        ['','P_loss', 'Q_loss', 'P_loss', 'Q_loss'], 
        ['P_lb, Ts='+str(P_lb_results[0]), PQ_losses[0], PQ_losses[1], '*', '*'],
        ['P_ub, Ts='+str(P_ub_results[0]), PQ_losses[2], PQ_losses[3], '*', '*'],
        ['Q_lb, Ts='+str(Q_lb_results[0]), PQ_losses[4], PQ_losses[5], '*', '*'],
        ['Q_ub, Ts='+str(Q_ub_results[0]), PQ_losses[6], PQ_losses[7], '*', '*']]

dash = '-' * 60
print(dash)
print('{:<15s}{:>20s}{:>5s}{:^18s}{:>4s}'.format(data[0][0],data[0][1],data[0][2],data[0][3], data[0][4]))
print('{:<10s}{:>12s}{:>12s}{:>12s}{:>12s}'.format(data[1][0],data[1][1],data[1][2],data[1][3], data[1][4]))
print(dash)
for i in range(len(data))[2:]:
    print('{:<12s}{:>12f}{:>12f}{:>12s}{:>12s}'.format(data[i][0],data[i][1],data[i][2],data[i][3], data[i][4]))


In [ ]:
# ----------- Figure 4, RHP on complex config 3 --------------
# Let's see where feas is lost! With complex config 3 (infeas)
all_act_locs = ['bus_611','bus_652','bus_645','bus_675',]
perf_nodes = ['bus_684','bus_684','bus_632','bus_671']
fig4_feas_configs, lzn_error_run_sum,heatMapNames = hm.runHeatMapProcess(fin_feeder, [], [], all_act_locs, perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
Source.from_file(heatMapNames[0]) # display graph in notebook
# we place 611-->684

In [ ]:
Source.from_file(heatMapNames[1]) # display graph in notebook
# we place 652-->684

In [ ]:
Source.from_file(heatMapNames[2]) # display graph in notebook
# we place 645-->632

In [ ]:
Source.from_file(heatMapNames[3]) # display graph in notebook
# we are looking to track 671, but stupidly choose 675 as act node even though heat map shows red
# we place 675-->671, resulting in infeas config

In [ ]:
# ------- Create Fig 5 and 7,+ validate fesa with eval_config ----------
# Desc: Two diagrams for 2 complex config1 and 2, labeled with act/perf node locations, plus PV curves

# Define Complex Config 3 (infeas)
all_act_locs = ['bus_675','bus_611','bus_652','bus_645']
perf_nodes = ['bus_671','bus_684','bus_684','bus_632']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# normally file_name='13NF_test.dot'
vis.markActuatorConfig(all_act_locs, fin_feeder, 'complex_config_1')
ff.clear_graph(fin_feeder)

# Define Complex Config 4 (feas)
all_act_locs = ['bus_611','bus_675','bus_645','bus_646']
perf_nodes = ['bus_671','bus_671','bus_632','bus_646']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
vis.markActuatorConfig(all_act_locs, fin_feeder, 'complex_config_2')
ff.clear_graph(fin_feeder)

# BUG: second call to markActCOnfig appends actuator nodes to previous graph

In [ ]:
# Figure 6: not done in python, simulate cc3 with lin state space sim and RTlab
# Figure 7: PV curves for cc3, produced at sam

In [ ]:
# ------ [Optional] run impedance-related functions --------
slack_bus = None
for bus_name, depth in depths.items():
    if depth == 0:
        slack_bus = bus_name
        break
print(depths)
# modify node names when change feeders
imp.plot_histogram_RX_ratios(fin_feeder,leaves,slack_bus,depths,leaves_only = True)
print('Z between buses:')
print(imp.get_total_impedance_between_two_buses(fin_feeder, 'bus_632', 'bus_684',depths))
print('\nZ to substation:')
print(imp.get_total_impedance_from_substation(fin_feeder, 'bus_632',depths))
print('\nX/R ratio of path to substation:')
print(imp.get_RX_ratio_tosubst(fin_feeder, 'bus_632',depths))
# should check on how to format the printing do that it's to like 2 decimal places

In [ ]:
# ----- run detLznRange ---------
z12 = imp.get_total_impedance_from_substation(fin_feeder, 'bus_632', depths) # 3 phase
B12 = np.zeros((3,3)) # TEMPORARY, line susceptance, Yshunt=G+jB
printCurves = True
all_act_locs = ['bus_611','bus_675','bus_645','bus_646']
lzn_err_max, slopes = lzn.detLznRange(fin_feeder, Vbase_ll, Sbase, z12, B12, all_act_locs, load_data, headerpath, substation_name, modelpath, depths,printCurves) # usually called by computeFeas
#^ calls computePQsweep, makePVcurve, makeQVcurve, computeLznItvl

In [ ]:
# ----- Run computeFParamSpace  and save R and X matrices as csv ---------

#importlib.reload(ctrl)
all_act_locs = ['bus_645','bus_646','bus_611','bus_675']
perf_nodes = ['bus_632','bus_646','bus_671','bus_671']
Fq_ub,Fp_ub=ctrl.computeFParamSpace_v2(fin_feeder, all_act_locs, perf_nodes,R,X,depths,node_index_map)
# this function prints Zgood (from R and X matrices) and Z_toSubst, where Z_toSubst is correct but doesnt match Zgood. Investigate
print('(Fp_ub,Fq_ub)=(',Fp_ub,',',Fq_ub,')')
#print('(q,p)=(',q,',',p,')')

# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
#np.savetxt('Rmat_13NF.csv', R, delimiter=',')
#np.savetxt('Xmat_13NF.csv', X, delimiter=',')

In [ ]:
# -------- Solve 3-ph fwd-bwd sweep for toy example ----------
# values have been validated wrt matlab translation
R12=np.identity(3) # per unit
X12=3*np.identity(3)
B12=(10**-6)*0.5*np.ones(3);
V1a=Vbase*np.cos(0*np.pi/180)+1j*Vbase*np.sin(0*np.pi/180); # angle=0 deg
V1b=Vbase*np.cos(-120*np.pi/180)+1j*Vbase*np.sin(-120*np.pi/180); # angle=-120 deg
V1c=Vbase*np.cos(+120*np.pi/180)+1j*Vbase*np.sin(+120*np.pi/180); # angle=+120 deg
V1=np.transpose([[V1a, V1b, V1c]])/Vbase;
V2=np.array([[0.97, 0.98, 0.985]]).transpose()
del2=np.array([[2, -120-1, 120-7]]).transpose() # degrees
V2cplx=np.multiply(V2,np.cos(del2*np.pi/180))+np.multiply(V2,np.sin(del2*np.pi/180)*1j)
I=np.dot(np.linalg.inv(R12+1j*X12),V1-V2cplx) # I=YV, 3x1 = (3x3)(3x1) matrix mult
S2=np.multiply(V2cplx,np.conj(I)); # 3x1=(3x1)(3x1) elementwise!
P2=np.real(S2)
Q2=np.imag(S2);

# inputs: 3x3 matrices and 3x1 vectors
V2_3ph,del2_3ph=lzn.solveFwdBwdSweep_2bus_3ph(R12, X12, B12,V1, P2, Q2)
# outputs: 3x1 vector for mag (Vpu), 3x1 vector for phase angle (degrees)
print('V2_3ph=',V2_3ph)
print('del2_3ph=',del2_3ph)

print('--------------------------------------------------')
R12=0.37878788*np.array([[ 0.3465,  0.156,  0.158,], # not pu
       [ 0.156,  0.3375,  0.1535,],
       [ 0.158,  0.1535,  0.3414]])
X12=0.37878788*np.array([[ 1.0179,  0.5017,  0.4236,],
       [ 0.5017,  1.0478,  0.3849,],
       [ 0.4236,  0.3849,  1.0348]])
B12=0.37878788*(10**-6)*np.array([[6.2998,  -1.9958,  -1.2595,],
    [-1.9958,  5.9597,  -0.7417,],
    [-1.2595,  -0.7417,  5.6386]]) # line susceptance, Yshunt=G+jB, impedacen excel gives in uS so multiply by 10^-6 to get to SI units
V1=Vbase*np.ones((3,1)) # 2401
# key: V1 in 3 phase is not all angle=0! Need cos and sin
V1a=Vbase*np.cos(0*np.pi/180)+1j*Vbase*np.sin(0*np.pi/180); # angle=0 deg
V1b=Vbase*np.cos(-120*np.pi/180)+1j*Vbase*np.sin(-120*np.pi/180); # angle=-120 deg
V1c=Vbase*np.cos(+120*np.pi/180)+1j*Vbase*np.sin(+120*np.pi/180); # angle=+120 deg
V1=np.transpose([[V1a, V1b, V1c]]);
P2=100000*np.ones((3,1)) # watts, to be congruent with volts and ohms
Q2=20000*np.ones((3,1))
V2_3ph,del2_3ph=lzn.solveFwdBwdSweep_2bus_3ph(R12, X12, B12,V1, P2, Q2)
print('V2_3ph=',V2_3ph)
print('del2_3ph=',del2_3ph)


# ----------- run makePVcurve ---------
Plb=20000 # 20kW
Pub=300000 # 300kW
Qlb=-5000 # -5kW
Qub=75000 # 75kW
printCurves=True
PQ_bounds=[Plb/Sbase, Pub/Sbase, Qlb/Sbase, Qub/Sbase] # input as pu
lzn.makePVcurve_3ph(PQ_bounds, Sbase, Vbase, R12, X12, B12, V1,printCurves)
lzn.makeQVcurve_3ph(PQ_bounds, Sbase, Vbase, R12, X12, B12, V1,printCurves)
# true curve looks fine, lzn curve wrong

In [ ]:
# ---------- Compare 1 to many on branch ----------
# Compare heatmaps for many actuators on a branch tracking middle node, and one colocated actuator/perf on middle node
branches = vis.assign_network_branches(fin_feeder, substation_name)
print('branches =', branches)

my_branch = branches[5] # choosing a branch that doesn't include substation (no interesting options for 13NF)
if 'bus_650' in my_branch:
    my_branch.remove('bus_650')
    my_branch.remove('bus_651')
middle_node = my_branch[len(my_branch)//2]
act_locs = my_branch
act_locs.remove(middle_node)
perf_nodes = [middle_node for _ in act_locs]
print('act locs =',act_locs)
print('perf nodes =',perf_nodes)
# add on "middle_node" pair, because heatmap is for once you've placed act at middle node
f, lzn_error_run_sum, one_act = hm.runHeatMapProcess(fin_feeder, [middle_node], [middle_node], [my_branch[2]], [middle_node], node_index_map, substation_name, depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
# display heatmap for one colocated actuator/perf node
ff.clear_graph(fin_feeder)
Source.from_file(one_act[-1])

In [ ]:
# ---------- Compare 1 to many on branch ----------
# add on "middle_node" pair, because heatmap is for once you've placed act at middle node
f, lzn_error_run_sum, many_acts = hm.runHeatMapProcess(fin_feeder, act_locs, perf_nodes, ['bus_633'], [middle_node], node_index_map, substation_name, depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
# display heatmap for many actuators (all along branch) tracking one perf node (on same branch)
print('perf nodes =',perf_nodes)
ff.clear_graph(fin_feeder)
Source.from_file(many_acts[-1]) # show the last file "many_acts"

In [ ]:
# ----- run markCommonFeasNodes ---------"
# input a list of feasible actuator configurations where each configuration is its own list (ie a list of lists)"
# for actuator locations that appear in every configuration, mark node blue"
# data is pulled from "fig1_feas_configs" assigned when creating fig 4
lst_feas_configs = []
for dic in fig4_feas_configs:
    lst_feas_configs += [dic['act']]
   
print('list feas configs=',lst_feas_configs)
shared_act_locs = vis.markCommonFeasNodes(lst_feas_configs, fin_feeder, file_name)
Source.from_file('shared_act_locs_' + file_name)

In [ ]:
# ----- run find_ActLoc_in_FeasConfigs ---------"
# act_loc = node name of potential actuator location"
# feas_configs = a list of feasible actuator configurations"
act_loc = 'bus_633'
configs_with_act = vis.find_ActLoc_in_FeasConfigs(act_loc, lst_feas_configs)
print('set of act locs assoc with feas configs that contain ',act_loc,':', configs_with_act)

In [ ]:
# ----- run plot_actuator_num_histogram ---------"
# input list of lists of feasible actuator configurations"
# output histogram reflecting number of actuators in each feas configuration"
vis.plot_actuator_num_histogram(lst_feas_configs, file_name)    

In [ ]:
# ----- run assign_network_branches ---------"
branches = vis.assign_network_branches(fin_feeder, substation_name)
vis.mark_network_branches(fin_feeder, branches, file_name, substation_name, depths)
Source.from_file('branch_key:' + file_name)

In [ ]:
# ----- run find_good_branches ---------"
# the 'best' branch is the branch with the most actuators on it across all configurations
num_good_branches = 3 # you choose
vis.find_good_branches(lst_feas_configs, branches, num_good_branches)

In [ ]:
# ----- run determine_good_or_bad_branch ---------"
print('Branches are ', branches)
branch = branches[-2]  # you choose as input
good_branch_threshold = 5 # threshhold # configs that much use the branch (i.e. an act on that branch) for the branch to be "good"
configs_with_branch = vis.determine_good_or_bad_branch(branch, lst_feas_configs, good_branch_threshold)
# output: configs that use the branch, and if that # exceeds threshhold, call the branch "good"
print('Num feas configs =', len(lst_feas_configs))

In [ ]:
# Rx ratio and phase coupling ratio informs bad branches
branches = vis.assign_network_branches(fin_feeder, substation_name)
for branch in branches:
    if 'bus_650'in branch:
        branch.remove('bus_650')
        branch.remove('bus_651')
rx_dic = {}
coupling_ratios = vis.phaseCouplingPerNode(fin_feeder, depths, file_name)
graphNodes_nosub = hm.remove_subst_nodes(fin_feeder, file_name) # dont consider substation nodes, node 650 and 651 for 13NF
for node_name in graphNodes_nosub:
    rx_3ph = (imp.get_RX_ratio_tosubst(fin_feeder, node_name, depths)).values()
    rx_dic[node_name] = sum(rx_3ph) 

normd_coupl_ratios = {}
normd_rx = {}
avg_coupl = np.mean(list(coupling_ratios.values()))
avg_rx = np.mean(list(rx_dic.values()))

for key, val in coupling_ratios.items():
    normd_coupl_ratios[key] = val/avg_coupl
for key, val in rx_dic.items():
    normd_rx[key] = val/avg_rx

mixed_vals = {}
for node in graphNodes_nosub:
    mixed_vals[node] = normd_coupl_ratios[node] + normd_rx[node]
vis.createColorMap(fin_feeder, mixed_vals, 'mixed_colormap')
Source.from_file('colorMap_mixed_colormap') # display colormap to see good/bad branches  

In [ ]:
# Rx ratio and phase coupling ratio informs bad branches cont.
# place an actuator on a 'bad' branch and run runHeatMapProcess
#'bad' branch has large phase coupling and/or rx values
sum_branches = {}
for i in range(len(branches)):
    branch = branches[i]
    # key = index of branch in branches list
    sum_branches[i] = 0
    for node in branch:
        sum_branches[i] += mixed_vals[node]

worst_branch_indx = max(sum_branches, key = sum_branches.get)
worst_mixdVal_branch = branches[worst_branch_indx]
set_act = [worst_mixdVal_branch[len(worst_mixdVal_branch)//2]]
set_perf = [worst_mixdVal_branch[len(worst_mixdVal_branch)//2]]
node_lst = list(graphNodes_nosub)
node_lst.remove(set_act[0])
test_act = [node_lst[0]]
test_perf = set_perf
feas_configs, lzn_error_run_sum, bad_branch_map = hm.runHeatMapProcess(fin_feeder, set_act, set_perf, test_act, test_perf, node_index_map, substation_name, depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

# display heatmap to show lots of infeas nodes 
Source.from_file(bad_branch_map[-1]) 

In [ ]:
# Rx ratio and phase coupling ratio informs bad branches cont.
# place an actuator on a 'good' branch and run runHeatMapProcess
#'good' branch has small phase coupling and/or rx values
best_branch_indx = min(sum_branches, key = sum_branches.get)
best_mixdVal_branch = branches[best_branch_indx]
set_act = [best_mixdVal_branch[len(best_mixdVal_branch)//2]]
set_perf = [best_mixdVal_branch[len(best_mixdVal_branch)//2]]
node_lst = list(graphNodes_nosub)
node_lst.remove(set_act[0])
test_act = [node_lst[0]]
test_perf = set_perf
feas_configs, lzn_error_run_sum, good_branch_map = hm.runHeatMapProcess(fin_feeder, set_act, set_perf, test_act, test_perf, node_index_map, substation_name, depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

# display heatmap to show lots of feas nodes 
Source.from_file(good_branch_map[-1])